<a href="https://colab.research.google.com/github/Pandu98-pkh/DeepLearning/blob/main/01.%20Week%201/Notebook%20Assigment/Regression%20Infrared%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regression Infrared Dataset**

## **1. Pemuatan dan Pemrosesan Data Awal**
### **1.1 Installasi**

In [92]:
!pip install torch
!pip install tensorflow
!pip install numpy pandas matplotlib scikit-learn

### **1.2 Import Library**

In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # untuk membagi data menjadi data latih dan data uji
from sklearn.preprocessing import LabelEncoder # untuk mengubah label kelas menjadi angka
from sklearn.metrics import mean_squared_error, r2_score
import math

### **1.3 Memuat Dataset**

In [94]:
url = "https://raw.githubusercontent.com/Pandu98-pkh/DeepLearning/refs/heads/main/01.%20Week%201/Dataset/Infrared.csv"

try:
  df = pd.read_csv(url)
except Exception as e:
  print(f"Error loading dataset: {e}")


In [95]:
df.sample(10) # menampilkan 10 sample data secara acak

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
316,Male,18-20,Asian,22.1,21.5,0.70,0.7275,35.0000,34.9225,34.4975,...,34.855000,34.7800,34.632500,34.7125,35.487500,35.137500,36.2225,36.1725,36.2225,37.19
79,Female,26-30,White,23.6,33.9,0.69,0.8250,35.7100,35.5125,34.9250,...,34.167500,34.3125,34.667500,34.5600,35.267500,35.040000,35.9700,35.2275,35.2625,36.29
482,Male,26-30,Asian,24.7,22.3,0.60,0.9725,35.6525,35.0375,34.0450,...,34.722500,34.3625,34.497500,34.6000,35.592500,35.415000,36.1825,36.1300,36.1650,36.79
995,Female,18-20,Black or African-American,23.5,16.7,0.60,0.7050,35.3175,35.5375,34.8150,...,34.552500,34.4600,34.685000,34.5600,35.805000,35.100000,35.8950,35.7200,35.7350,36.74
715,Female,21-25,White,24.1,18.5,0.60,0.8200,35.7900,35.6950,34.9575,...,34.910000,35.2900,34.905000,35.1850,35.942500,35.552500,36.2450,36.0925,36.1425,37.39
406,Male,18-20,White,24.1,47.8,0.58,1.2000,35.5675,35.4800,35.2050,...,34.933333,34.9600,34.753333,34.1900,35.263333,35.253333,35.9600,35.4800,35.5100,36.84
949,Male,26-30,Black or African-American,24.9,26.4,0.70,0.9000,35.5525,35.4525,34.8200,...,34.345000,34.0575,33.835000,34.0650,35.132500,34.805000,35.8400,35.7725,35.8200,37.09
870,Female,21-25,Multiracial,24.0,27.2,0.60,0.7775,34.9725,34.7675,34.5525,...,34.867500,34.9075,34.590000,34.5875,35.262500,35.235000,35.4875,34.9550,34.9900,36.44
294,Female,21-25,Asian,25.4,48.0,0.60,1.1850,35.9200,35.8550,35.6050,...,35.030000,35.2950,35.010000,35.4000,35.810000,35.795000,36.2400,36.1050,36.1400,37.39
623,Female,31-40,White,22.0,30.0,0.60,0.9725,35.7700,36.0200,34.8300,...,34.112500,34.2775,34.250000,34.7075,35.830000,35.020000,36.3100,35.9350,35.9725,37.24


In [96]:
df.describe()

,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,aveAllL13_1,T_RC1,T_RC_Dry1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
count,1020.000000,1020.000000,1018.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,...,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000
mean,24.115392,28.723039,0.729784,0.968648,35.596533,35.611474,34.888475,35.011345,35.659921,35.587143,...,34.567782,34.565340,34.487701,34.577293,35.421555,35.094144,36.084785,35.805519,35.838277,37.028382
std,1.336338,13.071627,2.456486,0.362587,0.574888,0.549760,0.718613,0.633836,0.553897,0.569278,...,0.669410,0.678663,0.668065,0.728678,0.523477,0.576045,0.491821,0.559258,0.559316,0.509502
min,20.200000,9.900000,0.540000,-0.590000,33.897500,34.122500,31.770000,32.902500,33.985000,33.825000,...,31.452500,31.657500,31.280000,31.150000,33.407500,32.440000,34.892500,33.802500,33.835000,35.540000
25%,23.400000,17.600000,0.600000,0.772500,35.247500,35.271875,34.456250,34.651250,35.332500,35.249375,...,34.180000,34.177500,34.096875,34.225000,35.116875,34.756458,35.775000,35.474375,35.500000,36.777500
50%,24.000000,26.300000,0.620000,0.940000,35.548750,35.575000,34.915000,34.997500,35.602500,35.533750,...,34.597500,34.602500,34.512917,34.622500,35.391250,35.100000,36.027500,35.790000,35.825000,36.940000
75%,24.700000,36.200000,0.700000,1.140000,35.872500,35.883125,35.300000,35.363125,35.910625,35.855625,...,34.968125,34.965000,34.878125,35.012500,35.675000,35.415000,36.280000,36.087500,36.120625,37.140000
max,29.100000,61.200000,79.000000,2.875000,38.405000,38.042500,37.575000,37.680000,38.385000,38.380000,...,37.075000,37.165000,37.212500,37.367500,38.002500,37.632500,38.807500,38.417500,38.455000,40.340000


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gender       1020 non-null   object 
 1   Age          1020 non-null   object 
 2   Ethnicity    1020 non-null   object 
 3   T_atm        1020 non-null   float64
 4   Humidity     1020 non-null   float64
 5   Distance     1018 non-null   float64
 6   T_offset1    1020 non-null   float64
 7   Max1R13_1    1020 non-null   float64
 8   Max1L13_1    1020 non-null   float64
 9   aveAllR13_1  1020 non-null   float64
 10  aveAllL13_1  1020 non-null   float64
 11  T_RC1        1020 non-null   float64
 12  T_RC_Dry1    1020 non-null   float64
 13  T_RC_Wet1    1020 non-null   float64
 14  T_RC_Max1    1020 non-null   float64
 15  T_LC1        1020 non-null   float64
 16  T_LC_Dry1    1020 non-null   float64
 17  T_LC_Wet1    1020 non-null   float64
 18  T_LC_Max1    1020 non-null   float64
 19  RCC1  

### **1.4 Mengonversi dan Menormalisasi Data**

In [98]:
# Mengubah data kategorikal menjadi angka
label_encoder = LabelEncoder()

# Mengubah label 'Gender' menjadi angka
df['Gender'] = label_encoder.fit_transform(df['Gender'])

# Mengubah label 'Age' menjadi angka
df['Age'] =label_encoder.fit_transform(df['Age'])

# Mengubah label 'Ethnicity' menjadi angka
df['Ethnicity'] = label_encoder.fit_transform(df['Ethnicity'])

# Memisahkan kolom yang digunakan sebagai fitur dan label
X = df.drop('aveOralM', axis=1).values # Mengasumsikan 'aveOralM' adalah kolom label
y = df['aveOralM'].values.reshape(-1, 1)

# menampilkan data fitur dan label
print(X, y)

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% data latih, 20% data uji

[[ 1.      5.      5.     ... 35.6925 35.635  35.6525]
 [ 0.      4.      2.     ... 35.175  35.0925 35.1075]
 [ 0.      2.      5.     ... 35.9125 35.86   35.885 ]
 ...
 [ 0.      0.      2.     ... 36.535  35.965  35.9975]
 [ 1.      3.      3.     ... 35.86   35.415  35.435 ]
 [ 0.      0.      5.     ... 35.9725 35.89   35.9175]] [[36.59]
 [37.19]
 [37.34]
 ...
 [37.59]
 [37.29]
 [37.19]]


### **1.5 Mempersiapkan Rumus Evaluasi (MSE, RMSE, RSquared)**

In [99]:


def evaluate_model(y_true, y_pred):
  """
  Evaluates the performance of a regression model using MSE, RMSE, and R-squared.

  Args:
    y_true: True target values.
    y_pred: Predicted target values.

  Returns:
    A dictionary containing MSE, RMSE, and R-squared values.
  """

  mse = mean_squared_error(y_true, y_pred)
  rmse = math.sqrt(mse)
  r2 = r2_score(y_true, y_pred)

  return {"MSE": mse, "RMSE": rmse, "R-squared": r2}


### **1.6 Membuat Fungsi Visualisasi**

In [100]:
# Fungsinya untuk menampilkan plot hasil prediksi
def plot_predictions(y_true, y_pred, model_name):
  """
  Plots predicted vs. actual values for a regression model.

  Args:
    y_true: True target values.
    y_pred: Predicted target values.
    model_name: Name of the model for the plot title.
  """
  plt.figure(figsize=(8, 6))
  plt.scatter(y_true, y_pred)
  plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red', linestyle='--')  # Add diagonal line
  plt.xlabel("Actual Values")
  plt.ylabel("Predicted Values")
  plt.title(f"Predicted vs Actual Values ({model_name})")
  plt.grid(True)
  plt.show()

# # Fungsinya untuk menampilkan plot hasil training model
def plot_training_history(history, model_name):
  """
  Plots the training history (e.g., loss over epochs) for a model.

  Args:
    history: The training history object (e.g., from TensorFlow/Keras).
    model_name: The name of the model for the plot title.
  """
  plt.figure(figsize=(10, 5))

  # Assuming history contains 'loss' and 'val_loss' keys
  if 'loss' in history.history:
    plt.plot(history.history['loss'], label='Training Loss')
  if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')

  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.title(f"Training History ({model_name})")
  plt.legend()
  plt.grid(True)
  plt.show()


## **2. Deep Learning dengan PyTorch**

### **2.1 Import Library**

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim

### **2.2 Convert data to PyTorch tensors**

In [102]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


### **2.3 Susun Model Neural Network Sederhana**

In [103]:
# prompt: Susun Model Neural Network Sederhana

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 128  # Number of neurons in the hidden layer
output_size = 1  # Output size (regression)
learning_rate = 0.01
num_epochs = 1000

# Initialize the model, loss function, and optimizer
model = SimpleNN(input_size, hidden_size, output_size)


In [104]:
criterion = nn.MSELoss()  # Mean Squared Error loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [105]:
# prompt: Training Model

# Training loop
history = pd.DataFrame() # to store the loss values for plotting later
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, loss={loss.item()}')

    # Print progress
    history = pd.concat([history, pd.DataFrame({'epoch': [epoch], 'loss': [loss.item()]})], ignore_index=True)

Epoch 0, loss=nan
Epoch 100, loss=nan
Epoch 200, loss=nan
Epoch 300, loss=nan
Epoch 400, loss=nan
Epoch 500, loss=nan
Epoch 600, loss=nan
Epoch 700, loss=nan
Epoch 800, loss=nan
Epoch 900, loss=nan
